# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [ ]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('/Users/Daniah/Downloads/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003_extracted/CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'],encoding='utf8')


training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {
        "word": token,
        "pos": pos,
        # "word_length": len(token),
        # "is_upper": token[0].isupper(),
        # "is_title": token.istitle(),
        # "has_digit": any(char.isdigit() for char in token),
    }
    
    training_features.append(a_dict)     
    training_gold_labels.append(ne_label)  


print(training_features[:6])
print(training_gold_labels[:6])
   

[{'word': 'EU', 'pos': 'NNP'}, {'word': 'rejects', 'pos': 'VBZ'}, {'word': 'German', 'pos': 'JJ'}, {'word': 'call', 'pos': 'NN'}, {'word': 'to', 'pos': 'TO'}, {'word': 'boycott', 'pos': 'VB'}]
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O']


In [12]:

train = ConllCorpusReader('/Users/Daniah/Downloads/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003_extracted/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in train.iob_words():
    a_dict = {
        "word": token,
        "pos": pos,
        
    }
    test_features.append(a_dict)        
    test_gold_labels.append(ne_label) 


print(test_features[:6])
print(test_gold_labels[:6])


[{'word': 'SOCCER', 'pos': 'NN'}, {'word': '-', 'pos': ':'}, {'word': 'JAPAN', 'pos': 'NNP'}, {'word': 'GET', 'pos': 'VB'}, {'word': 'LUCKY', 'pos': 'NNP'}, {'word': 'WIN', 'pos': 'NNP'}]
['O', 'O', 'B-LOC', 'O', 'O', 'O']


**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [ ]:
from collections import Counter

train = ConllCorpusReader('/Users/Daniah/Downloads/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003_extracted/CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'], encoding='utf8')
test = ConllCorpusReader('/Users/Daniah/Downloads/ba-text-mining-master/ba-text-mining-master/lab_sessions/lab4/CONLL2003_extracted/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'], encoding='utf8')

# Extract NERC labels for train and test
train_labels = [label for _, _, label in train.iob_words()]
test_labels = [label for _, _, label in test.iob_words()]

train_instances = len(train_labels)
test_instances = len(test_labels)

train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)


print(f"Training Data Statistics:")
print(f"Total instances: {train_instances}")
print(f"Frequency of NERC labels: {train_label_counts}")
print()

print(f"Test Data Statistics:")
print(f"Total instances: {test_instances}")
print(f"Frequency of NERC labels: {test_label_counts}")
print()


# I think there is a problem here because the labels are imbalanced??
# O -> no name entity

Training Data Statistics:
Total instances: 203621
Frequency of NERC labels: Counter({'O': 169578, 'B-LOC': 7140, 'B-PER': 6600, 'B-ORG': 6321, 'I-PER': 4528, 'I-ORG': 3704, 'B-MISC': 3438, 'I-LOC': 1157, 'I-MISC': 1155})

Test Data Statistics:
Total instances: 46435
Frequency of NERC labels: Counter({'O': 38323, 'B-LOC': 1668, 'B-ORG': 1661, 'B-PER': 1617, 'I-PER': 1156, 'I-ORG': 835, 'B-MISC': 702, 'I-LOC': 257, 'I-MISC': 216})



Discussing the balance of the dataset:(bullets points only need to write still)

If labels have similar counts --> the dataset is balanced.

if some labels are mor than the others --> the dataset is imbalanced.

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [ ]:

from sklearn.feature_extraction import DictVectorizer


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

vec = DictVectorizer()
all_features = training_features + test_features

the_array = vec.fit_transform(all_features)  
print("The new array:", the_array)


The new array: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 500112 stored elements and shape (250056, 27361)>
  Coords	Values
  (0, 20)	1.0
  (0, 7806)	1.0
  (1, 40)	1.0
  (1, 24281)	1.0
  (2, 14)	1.0
  (2, 8749)	1.0
  (3, 19)	1.0
  (3, 17663)	1.0
  (4, 33)	1.0
  (4, 26336)	1.0
  (5, 35)	1.0
  (5, 17485)	1.0
  (6, 14)	1.0
  (6, 6242)	1.0
  (7, 19)	1.0
  (7, 21607)	1.0
  (8, 6)	1.0
  (8, 178)	1.0
  (9, 20)	1.0
  (9, 12886)	1.0
  (10, 20)	1.0
  (10, 6023)	1.0
  (11, 20)	1.0
  (11, 5630)	1.0
  (12, 9)	1.0
  :	:
  (250043, 26194)	1.0
  (250044, 9)	1.0
  (250044, 1670)	1.0
  (250045, 20)	1.0
  (250045, 16161)	1.0
  (250046, 20)	1.0
  (250046, 7216)	1.0
  (250047, 14)	1.0
  (250047, 27215)	1.0
  (250048, 19)	1.0
  (250048, 26092)	1.0
  (250049, 13)	1.0
  (250049, 27230)	1.0
  (250050, 27)	1.0
  (250050, 20863)	1.0
  (250051, 15)	1.0
  (250051, 27345)	1.0
  (250052, 19)	1.0
  (250052, 17567)	1.0
  (250053, 5)	1.0
  (250053, 166)	1.0
  (250054, 20)	1.0
  (250054, 6058)	1.0
  (

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [ ]:
from sklearn import svm

In [ ]:
lin_clf = svm.LinearSVC()

In [ ]:
##### [ YOUR CODE SHOULD GO HERE ]
# lin_clf.fit( # your code here

**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [ ]:
# your code here

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [ ]:
import pandas

In [ ]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook